## <b>Proyecto final</b>
##  <center>"Análisis de clasificación de la transparencia de copa en <html><i>Pseudotsuga menziesii</i></html> con técnicas de aprendizaje automatizado" </center>
## <b>COA-501</b> Herramientas de cómputo para investigadores (R y Python)
## <b>Alumno:</b> Iván Fermín Quiroz Ibáñez
### <b>Correo:</b> <quiroz.ivan@colpos.mx>
### <b>Github</b>: <https://github.com/IFQI91/Proyecto_final_COA501_IFQI.git>
### <b>Dat Linux</b>: <https://datlinux.com>

<img src="./Imagenes/ml.jpg"/>

In [ ]:
#directorio de trabajo de binder local
getwd()

In [1]:
#cargar base de datos
base <- read.csv("/home/jovyan/Bases/ABCPE_MIXTO_B1_R_coordenadas.csv")
base$Arbol <- as.factor(base$Arbol)
base$Bloque <- as.factor(base$Bloque)
base$Anio <- as.factor(base$Anio)
base$Muestreo <- as.factor((base$Muestreo))
base$Fungicida<- as.factor((base$Fungicida))
base$Nutrimento <- as.factor((base$Nutrimento))
base$DDA <- as.factor((base$DDA))
base$trat <- as.factor((base$trat))
base$AltRan <- as.factor((base$AltRan))

In [3]:
#Base de datos dinámica
library(DT)
datatable(base,
          class="cell border stripe",
          width="600px",filter="top",
          caption="Base de datos")

HTML widgets cannot be represented in plain text (need html)

### **0) Estudio de caso**
Se cuenta con la base de datos de un experimento realizado en Puebla, dónde se evaluaron los efectos de fertilizantes y plaguicidas contra una enfermedad foliar
conocida como tizón suizo en árboles de navidad de la especie Douglas-fir. Se tomaron
variables como transparencia de copa, severidad, incidencia, longitud de brotes, área de acículas, etc. El objetivo del análisis es determinar la mejor técnica de machine learning o aprendizaje automático (Naive Bayes, K-NN y Random Forest) para clasificar la transparencia de copa en Douglas-fir.

<img src="./Imagenes/tc.jpg"/>

**Variables:**

- Arbol= unidad experimental (UE)
- Bloque= Transparencia de copa asignada como baja,media y alta
- Longitud=  distancia en grados, minutos y segundos que hay con respecto al meridiano principal, que es el meridiano de Greenwich (0º) de la UE
- Latitud= distancia en grados, minutos y segundos que hay con respecto al paralelo principal, que es el ecuador (0º)
- X_UTM= coordenada X en el sistema de coordenadas universal transversal de Mercator
- Y_UTM= coordenada Y en el sistema de coordenadas universal transversal de Mercator
- Altitud= distancia vertical de la UE respecto al nivel del mar
- AltRan= Rango de Altitud de las UE en dos categorías: Alta y Baja
- Fungicida= Tratamiento fungicida (Procloraz, Propiconazol o sin fungicida)
- Nutrimento= Tratamiento fertilizante (Urea, Sulfato de potasio o sin fertilizante)
- Rep= repetición de la UE por tratamiento en un diseño experimental factorial en bloques generalizados
- Muestreo= número de muestreo de follaje (se realizaron 6)
- DDA= días despúes de la aplicación o del establecimiento del experimento
- Anio= edad del follaje o muestra
- AcicR= número de acículas retenidas de rama o muestra
- Abs= Número de cicatrices de acículas de la rama o muestra
- TotalAc= Total de acículas (AcicR + Abs)
- Inc= Incidencia del tizón suizo
- Sevmed= Severidad media del tizón suizo
- Sevmed= Severidad mínima del tizón suizo
- Sevmax= Severidad máxima del tizón suizo
- LonBrot= Longitud de la rama o muestea en centímetros
- indcol= Índice de colonización (Inc * Sevmed)
- Afmed= Área media de acícula (Área de acículas en centímetros cuadrados por muestra)
- Afmax= Área máxima de acícula por muestra
- Afmin= Área mínima de acícula por muestra
- Aftotal=  Suma de área de acículas por muestra
- CA= Abs expresada en porcentaje (Abs*100/TotalAC)
- RA= RA expresada en porcentaje (RA*100/TotalAC)
- Color= Categorías de color por cada muestra (Verde claro, verde oscuro y rojizo)
- trat= Interacción de tratamientos (Fungicida*Nutrimento)
- ABCPEIC= Área bajo la curva progreso de la enfermedad con el índice de colonización

### **1) Base de datos y AED**

In [ ]:
options(warn=-1)
#base de datos
summary(base)
class(base)
str(base)

#matriz de diagramas de dispersion
pairs(base[,c(5,6,7,9,10,12,18,19,22,24,28:30,32)])

library(ggplot2)
library(GGally)
ggpairs(base[,c(5,6,7,18,19,22,24,28:30)], 
        title="correlograma con ggpairs") 

library(psych)
psych::pairs.panels(base[,c(5,6,7,18,19,22,24,28:30)])

In [ ]:
summary(base$Altitud)
2994-2973

In [ ]:
#Análisis de componentes principales 
base.pc<- na.omit(base[,c(14,2,5,6,7,8,18,19,22,24,28:30)])
pca <- prcomp(base.pc[,-c(1,2,6)], scale = T, center=T)
summary(pca)

#biplot
library(ggplot2)
Edad <- as.factor(base.pc$Anio)
ggplot(base.pc, aes(x = pca$x[,1], y = pca$x[,2], colour = Edad)) +
  geom_point(size=3) + xlab("PC1") + ylab("PC2")+
  ggtitle("PCA por Edad del follaje")

#biplot2
library(ggplot2)
ggplot(base.pc, aes(x = pca$x[,1], y = pca$x[,2], colour = Altitud)) +
  geom_point(size=3) + xlab("PC1") + ylab("PC2")+
  ggtitle("PCA por rango de altitud (2994-2973 msnm)")

#biplot3
library(ggplot2)
Alt <- as.factor(base.pc$AltRan)
ggplot(base.pc, aes(x = pca$x[,1], y = pca$x[,2], colour = Alt)) +
  geom_point(size=3) + xlab("PC1") + ylab("PC2")+
  ggtitle("PCA por AltRan")

In [ ]:
plot(pca,main="Sedimentación")
biplot(pca, main="PCA biplot")

colores <- function(vec){
  # la funci?n rainbow() devuelve un vector que contiene el n?mero de colores distintos
  col <- rainbow(length(unique(vec)))
  return(col[as.numeric(as.factor(vec))])
}

plot(pca$x[,c(1, 2)], col = colores(base$Bloque),
     pch = 19, 
     xlab = "PC1", 
     ylab = "PC2", main="PCA por Transparencia")

**Interpretación:** De acuerdo con el AED y ACP se seleccionaron las variables transparencia de copa, coordenadas UTM (X y Y), altitud, incidencia, severidad, longitud de brote, área de acícula, acículas caidas, acículas retenidas y color de acícula, para implementar las técnicas de ML.


### **2) Random Forest**

In [ ]:
options(warn=-1)
library(randomForest)
library(caret)

#Getting Data
base_rf <- na.omit(base[,c(2,5,6,7,18,19,22,24,28:30)])

base_rf$Bloque <- as.factor(base_rf$Bloque)
table(base_rf$Bloque)

#Data Partition
set.seed(123)
ind <- sample(2, nrow(base_rf), replace = TRUE, prob = c(0.7, 0.3))
train <- base_rf[ind==1,]
test <- base_rf[ind==2,]

#Random Forest in R
rf <- randomForest(Bloque~., data=train, proximity=TRUE) 
rf

#Confusion Matrix and Statistics
p1 <- predict(rf, train)
confusionMatrix(p1, train$Bloque)
(tab1 <- table(p1, train$Bloque))
1 - sum(diag(tab1)) / sum(tab1)
#error del 0%


p2 <- predict(rf, test)
confusionMatrix(p2, test$Bloque)
(tab2 <- table(p2, test$Bloque))
1 - sum(diag(tab2)) / sum(tab2)
#error del 0%



#Error rate of Random Forest
plot(rf)

#Tune mtry (Número de variables aleatorias utilizadas en cada árbol)
t <- tuneRF(train[,-1], train[,1],
            stepFactor = 0.5,
            plot = TRUE,
            ntreeTry = 5,
            trace = TRUE,
            improve = 0.05)
#mtry=6

#No. of nodes for the trees
hist(treesize(rf),
     main = "No. of Nodes for the Trees",
     col = "blue")
#media de 60 árboles

#Variable Importance
varImpPlot(rf,
           sort = T,
           n.var = 10,
           main = "Top 10 - Importancia de Variables")
importance(rf)

#Partial Dependence Plot
partialPlot(rf, train, Y_UTM, "Alta")

**Interpretación:** se obtuvo una precisión del 98% y un valor $\kappa$= 0.98, una
mtry (Número de variables aleatorias utilizadas en cada árbol) de 6, 60 nodos promedio por árbol, las variables de mayor peso son las coordenadas UTM (X y Y).

### **3) Naive Bayes**

In [ ]:
options(warn=-1)
#clasificacion en datos poco correlacionados
#https://www.r-bloggers.com/2021/04/naive-bayes-classification-in-r/
#Para segmentar imagenes (severidad)
#https://plantcv.readthedocs.io/en/latest/tutorials/machine_learning_tutorial/

library(dplyr)
library(ggplot2)


base_nb <- na.omit(base[,c(2,5,6,7,18,19,22,24,28:30)])

#Dplyr
base_nb$Bloque <- as.factor(base_nb$Bloque)
table(base_nb$Bloque)

base_nb %>%
  ggplot(aes(x=base_nb$Bloque,
  y=base_nb$Sevmed, fill = base_nb$Bloque)) +
  geom_boxplot() +theme_bw()+stat_summary(fun="mean")+
  ggtitle("Box Plot: Severidad por Transparencia")+
  xlab("Transparencia de copa") + ylab("Severidad")+
  theme(legend.position="none")


In [ ]:
library(naivebayes)
library(lattice)
#particion de datos
set.seed(1234)
ind <- sample(2, nrow(base_nb), replace = T, prob = c(0.7, 0.3))
train_nb <- base_nb[ind == 1,]
test_nb <- base_nb[ind == 2,]

nb <- naive_bayes(Bloque ~ ., data = train_nb, usekernel = T) 
nb 

plot(nb) 

**Interpretación:** se obtuvo una precisión de 49% y un valor de $\kappa$=0.23.
En general esta técnica no fue tan buena para clasificar la transparencia de copa.


### **4) K-NN**

In [ ]:
#https://rpubs.com/JairoAyala/601703

options(warn=-1)
library(kknn)

base_knn <- na.omit(base[,c(2,5,6,7,9,10,12,18,19,22,24,28:30,32)])

base_knn$Bloque <- as.factor(base_knn$Bloque)
table(base_knn$Bloque)


set.seed(2020)
muestra <- sample(1:624, 437)
train_knn <- base_knn[muestra,]#70%
test_knn<- base_knn[-muestra,]#30%
dim(train_knn)[1]
dim(test_knn)[1]

knn <- train.kknn(Bloque~ ., data = train_knn, kmax = 9)
knn

entre <- predict(knn, train_knn[,-1])
tt  <- table(train_knn[,1],entre)
tt

precision <- (sum(diag(tt)))/sum(tt)
precision

#precisión del 100 % en datos de entrenamiento


#Precisión test de prueba
pred    <- predict(knn, test_knn[,-1])
table   <- table(test_knn[,1],pred)
table


clas    <- (sum(diag(table)))/sum(table)
clas

#Precisión del 53% de datos de prueba

#matriz de confusion con la prueba

library(caret)
confusionMatrix(pred,test_knn$Bloque)

**Interpretación:** se obtuvo una precisión de 53% y un valor de $\kappa$=0.30.
En general esta técnica no fue tan buena para clasificar la transparencia de copa.

### **5) Curvas Receiver Operating Characteristic (ROC)**

In [ ]:
#Random forest
# Validation set assessment #2: ROC curves and AUC
# Needs to import ROCR package for ROC curve plotting:
library(ROCR)
# Calculate the probability of new observations belonging to each class
# prediction_for_roc_curve will be a matrix with dimensions data_set_size x number_of_classes
prediction_for_roc_curve <- stats::predict(rf,test[,-1],type="prob")
# Use pretty colours:
pretty_colours <- c("#F8766D","#00BA38","#619CFF")
# Specify the different classes 
test$Bloque <- as.factor(test$Bloque)
classes <- levels(test$Bloque)
# For each class
for (i in 1:3)
{
  # Define which observations belong to class[i]
  true_values <- ifelse(test[,1]==classes[i],1,0)
  # Assess the performance of classifier for class[i]
  pred <- prediction(prediction_for_roc_curve[,i],true_values)
  perf <- performance(pred, "tpr", "fpr")
  if (i==1)
  {
    plot(perf,main="ROC Curve-Random Forest",col=pretty_colours[i]) 
  }
  else
  {
    plot(perf,main="ROC Curve- Random Forest",col=pretty_colours[i],add=TRUE)
    abline (a = 0, b = 1, lty="dotted", lwd=2) 
  }
  # Calculate the AUC and print it to screen
  auc.perf <- performance(pred, measure = "auc")
  print(auc.perf@y.values)
}


#Naive Bayes (ROC)

# Validation set assessment #2: ROC curves and AUC
# Needs to import ROCR package for ROC curve plotting:
library(ROCR)
# Calculate the probability of new observations belonging to each class
# prediction_for_roc_curve will be a matrix with dimensions data_set_size x number_of_classes
prediction_for_roc_curve <- stats::predict(nb,test_nb[,-1],type="prob")
# Use pretty colours:
pretty_colours <- c("#F8766D","#00BA38","#619CFF")
# Specify the different classes 
test_nb$Bloque <- as.factor(test_nb$Bloque)
classes <- levels(test_nb$Bloque)
# For each class
for (i in 1:3)
{
  # Define which observations belong to class[i]
  true_values <- ifelse(test_nb[,1]==classes[i],1,0)
  # Assess the performance of classifier for class[i]
  pred <- prediction(prediction_for_roc_curve[,i],true_values)
  perf <- performance(pred, "tpr", "fpr")
  if (i==1)
  {
    plot(perf,main="ROC Curve-Naive Bayes",col=pretty_colours[i]) 
  }
  else
  {
    plot(perf,main="ROC Curve-Naive Bayes",col=pretty_colours[i],add=TRUE)
    abline (a = 0, b = 1, lty="dotted", lwd=2) 
  }
  # Calculate the AUC and print it to screen
  auc.perf <- performance(pred, measure = "auc")
  print(auc.perf@y.values)
}


#KNN (ROC)

# Validation set assessment #2: ROC curves and AUC
# Needs to import ROCR package for ROC curve plotting:
library(ROCR)
# Calculate the probability of new observations belonging to each class
# prediction_for_roc_curve will be a matrix with dimensions data_set_size x number_of_classes
prediction_for_roc_curve <- stats::predict(knn,test_knn[,-1],type="prob")
# Use pretty colours:
pretty_colours <- c("#F8766D","#00BA38","#619CFF")
# Specify the different classes 
test_knn$Bloque <- as.factor(test_knn$Bloque)
classes <- levels(test_knn$Bloque)
# For each class
for (i in 1:3)
{
  # Define which observations belong to class[i]
  true_values <- ifelse(test_knn[,1]==classes[i],1,0)
  # Assess the performance of classifier for class[i]
  pred <- prediction(prediction_for_roc_curve[,i],true_values)
  perf <- performance(pred, "tpr", "fpr")
  if (i==1)
  {
    plot(perf,main="ROC Curve - KNN",col=pretty_colours[i]) 
  }
  else
  {
    plot(perf,main="ROC Curve -KNN",col=pretty_colours[i],add=TRUE)
    abline (a = 0, b = 1, lty="dotted", lwd=2) 
  }
  # Calculate the AUC and print it to screen
  auc.perf<- performance(pred, measure = "auc")
  print(auc.perf@y.values)
}

**Interpretación:** De acuerdo la evaluación de las curvas ROC, es evidente qu el mejor modelo para clasificar a la transparencia de copa de Douglas-fir fue Random Forest, ya que tuvo mayor cantidad de positivos verdaderos en comparación con Naive Bayes y K-NN, donde se pueden considerar que son modelos de regulares a malos.


### **6) Comparación de métricas para validación**

In [ ]:
p <-rbind(c(0.98,0.49,0.53)) 
k <- rbind(c(0.98,0.23,0.30))
r <- rbind(c(1.00, 0.70, 0.72))
pres <- rbind(p,k,r)
pres <- as.data.frame(pres)
colnames(pres) <- c("Random Forest","Naive Bayes", "K-NN")
pres <- cbind(Métrica=c("Precisión(%)","Kappa", "AUC-ROC"),pres)
pres

### **7) Conclusiones**

De acuerdo la evaluación de las curvas ROC, precisión y valor de kappa, es evidente que el mejor modelo para clasificar a la transparencia de copa de Douglas-fir fue Random Forest, además tuvo mayor cantidad de positivos verdaderos con sensibilidad, especificidad y precisión equilibrada en comparación de Naive Bayes y K-NN, donde se pueden considerar que son modelos de regulares a malos para clasificar la transparencia de copa.

## **Bibliografía**

- finnstats. Naive Bayes Classification in R | R-Bloggers. 9 de abril de 2021, https://www.r-bloggers.com/2021/04/naive-bayes-classification-in-r/

- finnstats. Random Forest in R | R-Bloggers. 13 de abril de 2021, https://www.r-bloggers.com/2021/04/random-forest-in-r/

- Gandhi, Rohith. Naive Bayes Classifier. Medium, 17 de mayo de 2018, https://towardsdatascience.com/naive-bayes-classifier-81d512f50a7c  

- K-NN Classifier in R Programming». GeeksforGeeks, 18 de junio de 2020, https://www.geeksforgeeks.org/k-nn-classifier-in-r-programming/ 

- Naive Bayes Classifier in R Programming. GeeksforGeeks, 18 de junio de 2020, https://www.geeksforgeeks.org/naive-bayes-classifier-in-r-programming/ 

- Quiroz, I. I. F. 2019. Tolerancia al tizón suizo en una plantación de árboles de navidad en Aquixtla, Puebla. Tesis de Maestría. Programa en Ciencias Forestales. Colegio de Postgraduados. 101 p http://colposdigital.colpos.mx:8080/jspui/handle/10521/3571

- RPubs - KNN. https://rpubs.com/JairoAyala/601703. Accedido 20 de noviembre de 2022
